In [5]:
!python -m pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.4 MB/s eta 0:00:0000:010:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nvidia-physicsnemo 1.0.1 requires torch>=2.0.0, which is not installed.
torch-harmonics 0.6.5 requires torch, which is not installed.
torch-harmonics 0.6.5 requires triton, which is not installed.
mlflow 2.21.1 requires Jinja2<4,>=2.11; platform_system != "Windows", which is not installed.
dgl 2.4.0+cu121 requires networkx>=2.1, which is not installed.
dgl 2.4.0+cu121 requires torch<=2.4.0, which is not installed.
scikit-image 0.25.2 requires networkx>=3.0, which is not installed.
nvidia-physicsnemo 1.0.1 requires setuptools>=67.6.0, but you have setuptools 65.5.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import numpy as np
import os

In [7]:
rawData = np.load(os.path.join('dataset/', "rawData.npy"), allow_pickle=True)

In [9]:
rawData.keys()

dict_keys(['x', 'edge_attr', 'edge_index', 'mass', 'para'])

In [12]:
rawData['x'].shape, rawData['edge_attr'].shape, rawData['edge_index'].shape

((101, 401, 1699, 3), (4954, 3), (2, 4954))

In [13]:
rawData['mass'].shape, rawData['para'].shape

((1699, 1), (101,))

In [20]:
rawData['edge_attr']

array([[ 0.027991  , -0.013205  ,  0.03094944],
       [-0.002198  ,  0.030036  ,  0.03011632],
       [-0.034207  , -0.009637  ,  0.03553858],
       ...,
       [-0.14918   , -0.1027406 ,  0.18113615],
       [-0.0221    ,  0.1496993 ,  0.15132181],
       [ 0.14479   , -0.0806596 ,  0.16574111]], shape=(4954, 3))

In [21]:
rawData['edge_index']

array([[   0,    0,    1, ..., 1698, 1698, 1698],
       [  27, 1158,   36, ..., 1118, 1124, 1125]], shape=(2, 4954))

In [22]:
position_mesh = np.loadtxt("dataset/meshPosition_all.txt")

In [25]:
position_mesh[0] - position_mesh[27], np.linalg.norm(position_mesh[0] - position_mesh[27])

(array([-0.02799103,  0.01320499]), np.float64(0.030949464959144343))

In [26]:
position_mesh = np.loadtxt("dataset/meshPosition_all.txt")
position_pivotal = np.loadtxt("dataset/meshPosition_pivotal.txt")

In [29]:
position_mesh.shape, position_pivotal.shape, position_mesh.repeat(5, 1).shape

((1699, 2), (256, 2), (1699, 10))

In [30]:
nodes_index = np.arange(5)
batch_mesh = nodes_index.repeat_interleave([3,3,3,3,3])
position_mesh_batch = position_mesh.repeat(5, 1)
position_pivotal_batch = position_pivotal.repeat(5, 1)
batch_pivotal = nodes_index.repeat_interleave(
    ([len(position_pivotal)] * 5)
)

AttributeError: 'numpy.ndarray' object has no attribute 'repeat_interleave'

In [ ]:

# From train.py
def forward(self, graph, position_mesh, position_pivotal):
    with autocast(enabled=self.C.amp):
        z = self.model.encode(
            graph.ndata["x"],
            graph.edata["x"],
            graph,
            position_mesh,
            position_pivotal,
        )
        x = self.model.decode(
            z, graph.edata["x"], graph, position_mesh, position_pivotal
        )
        loss = self.criterion(x, graph.ndata["x"])
        return loss

In [ ]:
# from MESH_REDUCE
def knn_interpolate(
        self,
        x: torch.Tensor,
        pos_x: torch.Tensor,
        pos_y: torch.Tensor,
        batch_x: torch.Tensor = None,
        batch_y: torch.Tensor = None,
        k: int = 3,
        num_workers: int = 1,
    ):
        with torch.no_grad():
            assign_index = torch_cluster.knn(
                pos_x,
                pos_y,
                k,
                batch_x=batch_x,
                batch_y=batch_y,
                num_workers=num_workers,
            )
            y_idx, x_idx = assign_index[0], assign_index[1]
            diff = pos_x[x_idx] - pos_y[y_idx]
            squared_distance = (diff * diff).sum(dim=-1, keepdim=True)
            weights = 1.0 / torch.clamp(squared_distance, min=1e-16)

        y = torch_scatter.scatter(
            x[x_idx] * weights, y_idx, 0, dim_size=pos_y.size(0), reduce="sum"
        )
        y = y / torch_scatter.scatter(
            weights, y_idx, 0, dim_size=pos_y.size(0), reduce="sum"
        )

        return y.float(), x_idx, y_idx, weights

    def encode(self, x, edge_features, graph, position_mesh, position_pivotal):
        x = self.encoder_processor(x, edge_features, graph)
        x = self.PivotalNorm(x)
        nodes_index = torch.arange(graph.batch_size).to(x.device)
        batch_mesh = nodes_index.repeat_interleave(graph.batch_num_nodes())
        position_mesh_batch = position_mesh.repeat(graph.batch_size, 1)
        position_pivotal_batch = position_pivotal.repeat(graph.batch_size, 1)
        batch_pivotal = nodes_index.repeat_interleave(
            torch.tensor([len(position_pivotal)] * graph.batch_size).to(x.device)
        )
        # find k nearest points from all nodes to each pivotal node
        x, _, _, _ = self.knn_interpolate(
            x=x,
            pos_x=position_mesh_batch,
            pos_y=position_pivotal_batch,
            batch_x=batch_mesh, # BS x 1699
            batch_y=batch_pivotal, # 256 x BS
        )

        return x

    def decode(self, x, edge_features, graph, position_mesh, position_pivotal):
        nodes_index = torch.arange(graph.batch_size).to(x.device)
        batch_mesh = nodes_index.repeat_interleave(graph.batch_num_nodes())
        position_mesh_batch = position_mesh.repeat(graph.batch_size, 1)
        position_pivotal_batch = position_pivotal.repeat(graph.batch_size, 1)
        batch_pivotal = nodes_index.repeat_interleave(
            torch.tensor([len(position_pivotal)] * graph.batch_size).to(x.device)
        )
        # find k nearest points from all pivotal nodes to each node
        x, _, _, _ = self.knn_interpolate(
            x=x,
            pos_x=position_pivotal_batch,
            pos_y=position_mesh_batch,
            batch_x=batch_pivotal,
            batch_y=batch_mesh,
        )

        x = self.decoder_processor(x, edge_features, graph)
        return x